## Import Dependencies 

In [1]:
import sys

sys.path.insert(0, "../MIR")

In [2]:
from src.main import *

In [3]:
load_index('./x.pickle')

Path does not point to a valid saved index.


In [4]:
corpus_index = construct_positional_indexes()

100%|██████████| 1572/1572 [00:04<00:00, 364.94it/s]


In [5]:
corpus_index.save_index('./x.pickle')

In [ ]:
corpus_index.

In [1]:
from __future__ import unicode_literals

import re
import sys
import multiprocessing

import xml.etree.ElementTree as ET
import untangle

from hazm import *
from typing import List, NewType
from tqdm import tqdm
import pickle

In [2]:
sys.setrecursionlimit(10 ** 6)